In [3]:
from src.data import OxfordPetDatasetLoader
from src.config import Config
from src.data import DataPreprocessor, DataAugmentor


config = Config()
config.USE_IMAGENET_NORM = True          # For transfer learning
config.PRESERVE_ASPECT_RATIO = True      # Better image quality
config.ENABLE_QUALITY_ENHANCEMENT = True # Image enhancement
config.NORMALIZATION_METHOD = 'imagenet' # Best for pretrained models

In [4]:

loader = OxfordPetDatasetLoader(
    data_dir="./data",  # Tùy chọn
    download=True,
    log_level='INFO'
)

INFO:src.data.dataset_loader:Initialized Oxford Pet Dataset Loader
INFO:src.data.dataset_loader:Data directory: ./data


In [5]:

train_ds, val_ds, test_ds = loader.create_train_val_test_splits(
    val_split=0.2,
    seed=42
)

INFO:src.data.dataset_loader:Creating train/validation/test splits...
INFO:src.data.dataset_loader:Checking dataset availability...
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Dataset already available.
INFO:src.data.dataset_loader:Loading raw dataset splits: ['train', 'test']
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Constructing tf.data.Dataset oxford_iiit_pet for split ['train', 'test'], from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Train set: 3,680 examples
INFO:src.data.dataset_loader:Test set: 3,669 examples
INFO:src.data.dataset_loader:Split sizes - Train: 2,944, Val: 736, Test: 3,669


In [6]:
train_ds.element_spec

{'file_name': TensorSpec(shape=(), dtype=tf.string, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [7]:
augmentor = DataAugmentor(
    config=config,
    prob_geo=0.7,
    prob_photo=0.8,
    prob_mixup=0.3,
    prob_cutout=0.2,
    prob_mosaic=0.3,
)

train_ds = augmentor.create_augmented_dataset(
    train_ds,
    augmentation_factor=3
)

val_ds = augmentor.create_augmented_dataset(
    val_ds,
    augmentation_factor=1
)

In [8]:
train_ds.element_spec

{'image': TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'segmentation_mask': TensorSpec(shape=(512, 512, 1), dtype=tf.float32, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'file_name': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [9]:
processor = DataPreprocessor(
    config=Config(),
    shuffle_buffer=5000,
)

In [10]:
train_ds = processor.create_training_dataset(
    train_ds,
    batch_size=8,
    task="detection",
    cache_filename="train_cache"  # File caching for speed
)

val_ds = processor.create_validation_dataset(
    val_ds,
    batch_size=8,
    task="detection"
)

Original dataset element spec:
{'image': TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None), 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None), 'segmentation_mask': TensorSpec(shape=(512, 512, 1), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None), 'file_name': TensorSpec(shape=(), dtype=tf.string, name=None)}


2025-06-16 17:32:11.750445: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-06-16 17:32:20.885561: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] fused(ShuffleDatasetV3:22,RepeatDataset:23): Filling up shuffle buffer (this may take a while): 3487 of 5000
2025-06-16 17:32:24.863618: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


Sample element type: <class 'dict'>
Sample keys: ['image', 'head_bbox', 'segmentation_mask', 'label', 'species', 'file_name']
Dataset element spec after preprocessing:
(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), {'bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None), 'pet_class': TensorSpec(shape=(), dtype=tf.int32, name=None), 'species': TensorSpec(shape=(), dtype=tf.int32, name=None), 'valid_bbox': TensorSpec(shape=(), dtype=tf.bool, name=None), 'area': TensorSpec(shape=(), dtype=tf.float32, name=None)})
Original dataset element spec:
{'image': TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None), 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None), 'segmentation_mask': TensorSpec(shape=(512, 512, 1), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None), 'file_name': TensorSpec(shape=(), dtype=tf.string, name=None)}
Sample element type: <class 'd